<a href="https://colab.research.google.com/github/skywalker0803r/base_ball_detect_lab/blob/main/%E6%8E%A8%E8%AB%96%E6%B8%AC%E8%A9%A6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import joblib
import os
import pickle
from sklearn.pipeline import Pipeline
import numpy as np

# 必要函數
def pad_sequence(seq, max_len):
    pad_len = max_len - len(seq)
    return seq + [[0,0,0,0]] * pad_len

def traj_vec_filter(traj_vec, aspect_ratio_threshold=0.33):
    """
    過濾不合理的軌跡框（非棒球的偵測框）。
    參數:
        traj_vec: List of [x1, y1, x2, y2]
        aspect_ratio_threshold: 當高寬比或寬高比 > 1 + threshold 時視為非正方形
    回傳:
        過濾後的軌跡
    """
    filtered = []
    for box in traj_vec:
        x1, y1, x2, y2 = box
        w = x2 - x1
        h = y2 - y1
        if w <= 0 or h <= 0:
            continue
        aspect_ratio = w / h
        if (1 - aspect_ratio_threshold) <= aspect_ratio <= (1 + aspect_ratio_threshold):
            filtered.append(box)
    return filtered

class TrajectoryPreprocessor:
    def __init__(self, max_len):
        self.max_len = max_len

    def transform(self, X_raw):
        out = []
        for traj in X_raw:
            traj_vec_raw = [list(t[1:]) if t[1] is not None else [0,0,0,0] for t in traj]
            traj_vec = traj_vec_filter(traj_vec_raw)
            traj_vec = pad_sequence(traj_vec, self.max_len)
            out.append(traj_vec)
        return np.array(out).reshape(len(out), -1)
def load_model(load_dir):
    """
    載入模型、LabelEncoder 和 Preprocessor，回傳推論用的 pipeline。
    回傳:
        pipeline_pitch, pipeline_desc, le_pitch_type, le_description
    """
    clf_pitch = joblib.load(os.path.join(load_dir, 'clf_pitch_type.pkl'))
    clf_desc = joblib.load(os.path.join(load_dir, 'clf_description.pkl'))

    with open(os.path.join(load_dir, 'le_pitch_type.pkl'), 'rb') as f:
        le_pitch = pickle.load(f)

    with open(os.path.join(load_dir, 'le_description.pkl'), 'rb') as f:
        le_desc = pickle.load(f)

    with open(os.path.join(load_dir, 'preprocessor_config.pkl'), 'rb') as f:
        config = pickle.load(f)

    # 必須事先定義 TrajectoryPreprocessor 類別
    preprocessor = TrajectoryPreprocessor(max_len=config['max_len'])

    pipeline_pitch = Pipeline([
        ('preprocess', preprocessor),
        ('rf', clf_pitch)
    ])

    pipeline_desc = Pipeline([
        ('preprocess', preprocessor),
        ('rf', clf_desc)
    ])

    return pipeline_pitch, pipeline_desc, le_pitch, le_desc

In [16]:
# 載入pkl檔案
pipeline_pitch, pipeline_desc, le_pitch, le_desc = load_model('Shohei_Ohtani_model')

# 載入推論資料
with open('/content/drive/MyDrive/Baseball Movies/Shohei_Ohtani_SL_videos_4S/baseball_trajectory_bbox_with_pitch_type_description/pitch_0001_baseball_trajectory_with_label.pkl', 'rb') as f:
    one_data = pickle.load(f)
# 推論
X_new = [one_data['trajectory']]
print("Pitch Type:", le_pitch.inverse_transform(pipeline_pitch.predict(X_new))[0])
print("Description:", le_desc.inverse_transform(pipeline_desc.predict(X_new))[0])
print("Description score:", pipeline_desc.predict_proba(X_new)[0][1])

# 載入推論資料
with open('/content/drive/MyDrive/Baseball Movies/Shohei_Ohtani_FS_videos_4S/baseball_trajectory_bbox_with_pitch_type_description/pitch_0001_baseball_trajectory_with_label.pkl', 'rb') as f:
    one_data = pickle.load(f)
# 推論
X_new = [one_data['trajectory']]
print("Pitch Type:", le_pitch.inverse_transform(pipeline_pitch.predict(X_new))[0])
print("Description:", le_desc.inverse_transform(pipeline_desc.predict(X_new))[0])
print("Description score:", pipeline_desc.predict_proba(X_new)[0][1])

# 載入推論資料
with open('/content/drive/MyDrive/Baseball Movies/Shohei_Ohtani_FF_videos_4S/baseball_trajectory_bbox_with_pitch_type_description/pitch_0001_baseball_trajectory_with_label.pkl', 'rb') as f:
    one_data = pickle.load(f)
# 推論
X_new = [one_data['trajectory']]
print("Pitch Type:", le_pitch.inverse_transform(pipeline_pitch.predict(X_new))[0])
print("Description:", le_desc.inverse_transform(pipeline_desc.predict(X_new))[0])
print("Description score:", pipeline_desc.predict_proba(X_new)[0][1])

Pitch Type: ST
Description: good
Description score: 0.82
Pitch Type: FS
Description: good
Description score: 0.73
Pitch Type: FF
Description: good
Description score: 0.81


In [17]:
# 載入pkl檔案
pipeline_pitch, pipeline_desc, le_pitch, le_desc = load_model('Gerrit_Cole_model')

# 載入推論資料
with open('/content/drive/MyDrive/Baseball Movies/Gerrit_Cole_CH_videos_4S/baseball_trajectory_bbox_with_pitch_type_description/pitch_0029_baseball_trajectory_with_label.pkl', 'rb') as f:
    one_data = pickle.load(f)
# 推論
X_new = [one_data['trajectory']]
print("Pitch Type:", le_pitch.inverse_transform(pipeline_pitch.predict(X_new))[0])
print("Description:", le_desc.inverse_transform(pipeline_desc.predict(X_new))[0])
print("Description score:", pipeline_desc.predict_proba(X_new)[0][1])

# 載入推論資料
with open('/content/drive/MyDrive/Baseball Movies/Gerrit_Cole_FF_videos_4S/baseball_trajectory_bbox_with_pitch_type_description/pitch_0001_baseball_trajectory_with_label.pkl', 'rb') as f:
    one_data = pickle.load(f)
# 推論
X_new = [one_data['trajectory']]
print("Pitch Type:", le_pitch.inverse_transform(pipeline_pitch.predict(X_new))[0])
print("Description:", le_desc.inverse_transform(pipeline_desc.predict(X_new))[0])
print("Description score:", pipeline_desc.predict_proba(X_new)[0][1])

# 載入推論資料
with open('/content/drive/MyDrive/Baseball Movies/Gerrit_Cole_SL_videos_4S/baseball_trajectory_bbox_with_pitch_type_description/pitch_0001_baseball_trajectory_with_label.pkl', 'rb') as f:
    one_data = pickle.load(f)
# 推論
X_new = [one_data['trajectory']]
print("Pitch Type:", le_pitch.inverse_transform(pipeline_pitch.predict(X_new))[0])
print("Description:", le_desc.inverse_transform(pipeline_desc.predict(X_new))[0])
print("Description score:", pipeline_desc.predict_proba(X_new)[0][1])

Pitch Type: CH
Description: bad
Description score: 0.12
Pitch Type: FF
Description: bad
Description score: 0.05
Pitch Type: SL
Description: bad
Description score: 0.12


In [18]:
# 載入pkl檔案
pipeline_pitch, pipeline_desc, le_pitch, le_desc = load_model('Yu_Darvish_model')

# 載入推論資料
with open('/content/drive/MyDrive/Baseball Movies/Yu_Darvish_FF_videos_4S/baseball_trajectory_bbox_with_pitch_type_description/pitch_0001_baseball_trajectory_with_label.pkl', 'rb') as f:
    one_data = pickle.load(f)
# 推論
X_new = [one_data['trajectory']]
print("Pitch Type:", le_pitch.inverse_transform(pipeline_pitch.predict(X_new))[0])
print("Description:", le_desc.inverse_transform(pipeline_desc.predict(X_new))[0])
print("Description score:", pipeline_desc.predict_proba(X_new)[0][1])

# 載入推論資料
with open('/content/drive/MyDrive/Baseball Movies/Yu_Darvish_FS_videos_4S/baseball_trajectory_bbox_with_pitch_type_description/pitch_0001_baseball_trajectory_with_label.pkl', 'rb') as f:
    one_data = pickle.load(f)
# 推論
X_new = [one_data['trajectory']]
print("Pitch Type:", le_pitch.inverse_transform(pipeline_pitch.predict(X_new))[0])
print("Description:", le_desc.inverse_transform(pipeline_desc.predict(X_new))[0])
print("Description score:", pipeline_desc.predict_proba(X_new)[0][1])

# 載入推論資料
with open('/content/drive/MyDrive/Baseball Movies/Yu_Darvish_SL_videos_4S/baseball_trajectory_bbox_with_pitch_type_description/pitch_0002_baseball_trajectory_with_label.pkl', 'rb') as f:
    one_data = pickle.load(f)
# 推論
X_new = [one_data['trajectory']]
print("Pitch Type:", le_pitch.inverse_transform(pipeline_pitch.predict(X_new))[0])
print("Description:", le_desc.inverse_transform(pipeline_desc.predict(X_new))[0])
print("Description score:", pipeline_desc.predict_proba(X_new)[0][1])

Pitch Type: FF
Description: bad
Description score: 0.14
Pitch Type: FS
Description: good
Description score: 0.79
Pitch Type: SL
Description: bad
Description score: 0.14
